In [1]:
import numpy as np
import pandas as pd
import easyocr as eo

In [2]:
print('hello world')

hello world


In [3]:
df = pd.read_csv('dataset/train_train_train.csv')

In [4]:
df.head()

,image_link,group_id,entity_name,entity_value,image_name
0,https://m.media-amazon.com/images/I/61Am7goxaF...,178778,depth,3.9 centimetre,6617892b-9ef8-40dc-a834-4623584e4c44.jpg
1,https://m.media-amazon.com/images/I/51R8AlSuVK...,347320,depth,100.0 inch,0088cdf4-5092-4780-81be-dc41cdb4e9c4.jpg
2,https://m.media-amazon.com/images/I/51KghPIV6J...,826444,depth,90.0 millimetre,858aaef5-ad30-420a-966b-2ae49c5b8e87.jpg
3,https://m.media-amazon.com/images/I/51gqMA7+K0...,494658,depth,53.0 centimetre,4586f615-0158-4cdd-8201-f61965bd71dd.jpg
4,https://m.media-amazon.com/images/I/51YDWb+0+m...,861555,depth,217.0 millimetre,334836d1-92b8-4110-8713-fbbf56907fad.jpg


In [5]:
df.columns

Index(['image_link', 'group_id', 'entity_name', 'entity_value', 'image_name'], dtype='object')

In [6]:
reader = eo.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/Users/sarthaktanwar/Coding/python-projects/Amazon-ML-2024/amazonenv/lib/python3.12/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.

In [7]:
image_folder = 'dataset/Training'

In [29]:
# Function to extract text from image
def extract_text_from_image(image_path):
    try:
        # Read the image and extract text
        simple_result = reader.readtext(image_path, detail=0)  # detail=0 gives just the text
        detailed_result = reader.readtext(image_path)  # full output

        # Convert all elements of detailed_result to string to avoid serialization issues
        detailed_result_serializable = [[str(item) for item in sublist] for sublist in detailed_result]

        return simple_result, detailed_result_serializable  # Return the list of extracted text
    except Exception as e:
        return str(e), str(e)

In [30]:
df['extracted_info'] = ""
df['full_info'] = ""

In [31]:
import os
import json

In [32]:
# Function to extract text from an image using the index in the CSV
def extract_text_by_index(index):
    if index < len(df):
        image_name = df.loc[index, 'image_name']  # Get the image name based on the index
        image_path = os.path.join('dataset/Training', image_name)  # Replace with your folder path
        
        # Extract text from the image
        result = reader.readtext(image_path, detail=0)  # detail=0 gives just the text
        
        # Print the extracted text
        print(f"Extracted Text from {image_name}: {result}")
    else:
        print("Index out of range.")

In [33]:
extract_text_by_index(25)

Extracted Text from 1a41dfc2-63dc-4af8-945e-1b3f65bac808.jpg: ['42"', '80.5"', '27.5"']


In [34]:
# Loop through the DataFrame and update the columns
for index, row in df.iterrows():
    image_path = os.path.join(image_folder, row['image_name'])
    
    # Extract text
    simple_result, detailed_result = extract_text_from_image(image_path)
    
    # Store the results in the corresponding columns
    df.at[index, 'extracted_text'] = simple_result  # Store simple text
    
    # Convert the detailed result to a JSON string to avoid Pandas storing issues
    df.at[index, 'extracted_text_with_details'] = json.dumps(detailed_result)  # Store detailed output as JSON


In [36]:

# Define the folder where you want to save the CSV
output_folder = 'dataset/'  # Replace with your desired output folder path

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Define the full path for the output CSV file
output_file = os.path.join(output_folder, 'train_train_train_train.csv')

# Save the updated DataFrame to the new CSV file in the specific folder
df.to_csv(output_file, index=False)

print(f"Text extraction complete! The results have been saved to {output_file}.")

Text extraction complete! The results have been saved to dataset/train_train_train_train.csv.


In [37]:
df.columns

Index(['image_link', 'group_id', 'entity_name', 'entity_value', 'image_name',
       'extracted_info', 'extracted_text', 'full_info',
       'extracted_text_with_details'],
      dtype='object')

In [38]:
df = df.drop(columns=['full_info'])
df = df.drop(columns=['extracted_info'])

In [39]:
df.columns

Index(['image_link', 'group_id', 'entity_name', 'entity_value', 'image_name',
       'extracted_text', 'extracted_text_with_details'],
      dtype='object')

In [40]:
output_file

'dataset/train_train_train_train.csv'

In [41]:
df.to_csv(output_file, index=False)